In [1]:
import numpy as np
import os, inspect
from imp import warnings, reload
import SPARCdata
reload(SPARCdata)
from SPARCdata import read_SPARC_data, dt, create_galaxies_arr, fill_galaxy, unpack_emcee_params
import SIDMmodel1
reload(SIDMmodel1)
from SIDMmodel1 import *
# import SIDMmodel_cm_am
# reload(SIDMmodel_cm_am)
# from SIDMmodel_cm_am import *
import autocorr
reload(autocorr)
from autocorr import get_autocorr_N
from matplotlib import pyplot as plt
%matplotlib inline

warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)

np.random.seed(1234)

script_dir = os.path.dirname(os.path.abspath(inspect.getframeinfo(inspect.currentframe()).filename))
ddir = 'Data_share'
dset = 'SPARC'
ddir = os.path.join(script_dir, '..', '..', '..', 'Projects', ddir, dset)
print(ddir)

#ddir = '/Users/mkapling/Dropbox/Data_Share/SPARC'

emcee_params = ndim, nwalkers, nburn, niter, nthin, nthreads =\
5, 40, 400, 2000, 5, 1

n_poly = 4 # regression polynomial order


ModuleNotFoundError: No module named 'SPARCdata'

In [2]:
import os, copy, pickle
from time import time
from numpy.lib.recfunctions import stack_arrays

data, dt_in = read_SPARC_data(ddir)
dt_galaxy, dt_blobs = dt(emcee_params, dt_in)
print(dt_blobs)
print(len(data))

start, end = (0, len(data))
g = create_galaxies_arr(data, start, end, dt_galaxy)    
for galaxy in g:
    fill_galaxy(ddir, galaxy, n_poly)


[('rho0', '<f8'), ('sigma0', '<f8'), ('cross', '<f8'), ('ml_disk', '<f8'), ('ml_bulge', '<f8'), ('r1', '<f8'), ('mass_r1', '<f8'), ('rho_r1', '<f8'), ('rhos', '<f8'), ('rs', '<f8'), ('vmax', '<f8'), ('rmax', '<f8'), ('mvir', '<f8'), ('rvir', '<f8'), ('cvir', '<f8'), ('slope_15pRvir', '<f8'), ('chisq', '<f8'), ('v_dm', 'O'), ('v_sm', 'O'), ('v_b', 'O'), ('v_d', 'O'), ('v_m', 'O')]
153
NGC0891 : changing Q value to 10 from  1
NGC7331 : changing Q value to 10 from  1
UGC01281 : changing Q value to 11 from  1


In [ ]:
# set cross section in cm**2/g
# set to None to vary
cross = 3 

# this multiplies Vflat and is added quadratically to the error in data set
abs_e_v_factor = 0.05 

result_for_start_pos_only = False
specify_start_pos = False
equilibrium_solution_without_baryons = False
only_bulges = False
no_bulges = True
use_name_to_select = True
gname = 'NGC6503'
save_output = False

file_name_store = 'sparc_cross_'+str(cross)
fig_dir = 'sidm-cross_'+str(cross)

if specify_start_pos:
    inp_p = input("rho0, sigma0, cross, ml_disk, ml_bulge")
else:
    inp_p = None

os.makedirs(fig_dir, exist_ok=True)
os.makedirs(os.path.join(fig_dir,'corner_figs'), exist_ok=True)
os.makedirs(os.path.join(fig_dir,'vrot_figs'), exist_ok=True)
os.makedirs(os.path.join(fig_dir,'trace_plots'), exist_ok=True)

ss = sidm_setup()
mn = match_nfw()

if inp_p:
    start_params = ss.pack(*[float(v) for v in inp_p.split(",")])
else:
    start_params = None

def get_m200_c200(vmax, rmax):
    rhos = vmax ** 2 * 2.163**3 / (ss.GNewton * ss.fourpi * 0.467676 * rmax ** 2 )            
    lgrhos = np.log(rhos)
    if lgrhos > mn._logrhos[-1]:
        c200 = mn._c[-1]*(lgrhos/mn._logrhos[-1])**11
    else:
        if lgrhos < mn._logrhos[0]:
            c200 = mn._c[0]
        else:
            c200 = mn._getc(lgrhos)
    mnfw0 = vmax**2 * rmax / ss.GNewton / mn.nfw_m_profile(2.163)
    m200 = mnfw0 * mn.nfw_m_profile(c200)
    return m200, c200, np.log10(c200/mn.dutton_c200(m200))

def get_priors_bounds(galaxy): 
    rmax_prior = False
    bulge_prior = False # enforces ml_bulge > ml_disk with strictness defined by bulge_prior_width
    bulge_prior_width = 0.05 # in ml_bulge-ml_disk
    log10rmax_spread = 0.11
    log10c200_spread = 0.11
    if sum(galaxy.Data.Bulge) > 0:
        tophat_prior = True
        tophat_width = 3
    else:
        tophat_prior = True
        tophat_width = 3
# cosmo and ml prior parameters: 
# (rmax_prior = True or False; if False use c200;
# rmax at 100 km/s in kpc, \
# slope = dlog(rmax)/dlog(vmax), \
# spread in P(log10(rmax)|vmax), \ 
# spread in P(log10(c200)|m200); only use if rmax_prior = False,\
# tophat_prior = True or False; if False then log-normal, \
# half_width in units of spread for tophat prior, \
# ml_median for disk, log10ml_spread, \
# bulge_prior = True or False; if True ML_bulge>ML_disk)
    ml_median = 0.5 # if ml_median < 0, then impose AM
    log10ml_spread = 100
    prior_params = (rmax_prior, 27, 1.4, log10rmax_spread,\
                    log10c200_spread, tophat_prior, tophat_width,\
                    ml_median, log10ml_spread, bulge_prior, bulge_prior_width)
    if galaxy.Vflat > 0: 
        vmax_prior = True
        abs_e_v = abs_e_v_factor*galaxy.Vflat
    else:
        vmax_prior = False
        abs_e_v = abs_e_v_factor*((0.5*galaxy.Lum+galaxy.MHI)*1e9/50)**0.25        
    reg_params = (abs_e_v, 0.0, vmax_prior, 1.414)
# log10(ss.rate_const * cross * rho0 * sigma0), log10(sigma0), \
# log10(cross/(cm^2/g)), ml_disk, ml_bulge   
    if cross is not None:
        bounds = [(np.log10(2),np.log10(1e5)),\
                  (np.log10(2),np.log10(500)),\
                  (np.log10(cross),np.log10(cross)),\
                  (0.3,0.8),\
                  (0.3,0.8)]
    else:
        bounds = [(np.log10(2),np.log10(1e5)),\
                  (np.log10(2),np.log10(500)),\
                  (np.log10(0.1),np.log10(10)),\
                  (0.3,0.8),\
                  (0.3,0.8)]
# Ren et al ML prior was 0.1 to 10
# Katz et al ML prior was 0.3 to 0.8
    return prior_params, reg_params, bounds

def mask(galaxies):
    m = [True]*len(galaxies)
#     m = [False if item.Vflat > 0 or len(item.Data.R) < 5 else True for item in galaxies]
    if use_name_to_select:
        m = [True if item.Galaxy == gname else False for item in galaxies]
    else: 
        if only_bulges and not no_bulges:
            m = [True if sum(item.Data.Bulge) > 0 else False for item in galaxies]
        elif no_bulges and not only_bulges:
            m = [False if sum(item.Data.Bulge) > 0 else True for item in galaxies]
        else:
            pass
    return m

galaxies = g[mask(g)]
print("number of galaxies to be run: ", len(galaxies))

for galaxy in galaxies:
    print(galaxy.Galaxy)

    prior_params, reg_params, bounds = get_priors_bounds(galaxy)
    ndim, nwalkers, nburn, niter, nthin, nthreads = unpack_emcee_params(emcee_params)
    
    if result_for_start_pos_only and start_params:
        bounds = list(zip(np.array(start_params)*0.999,\
                          np.array(start_params)*1.001))
    else:
        start_params = None
        
#     galaxy.Data.e_V *= 1e6
#     galaxy.Data.Disk *= np.sqrt( [ 3-2*np.tanh(x*0.5) for x in galaxy.Data.R ] )
    t = time()
    
#     mn = constrained_sidm_profile('planck13',cross,10.0,galaxy) # for the constrained run
    args = (galaxy, ss, mn, emcee_params, prior_params, reg_params, bounds)

    if start_params:
        start = np.array(start_params)
        std = [min(0.05, 0.1*(start[i]-bounds[i][0]), 0.1*(bounds[i][1]-start[i])) for i in range(len(start))]
    else:
        start, std = start_pos(args)

    pos0 = np.array([np.array(start)+ [np.random.uniform(-stdv,stdv) for stdv in std] \
                     for i in range(nwalkers)])
    assert pos0.shape == (nwalkers, ndim)

    sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, args=[args], threads=nthreads)
    for pos, lnprob0, state, blobs in sampler.sample(pos0, iterations=nburn): # burn in
        pass

    sampler.reset() 
    for i, result in enumerate(sampler.sample(pos,iterations=niter)):
        if (i+1)%100 == 0:
            print("{0:5.1%}".format(float(i) / niter), time()-t)

#    sampler.run_mcmc(pos, niter)

    assert sampler.chain.shape == (nwalkers, niter, ndim)    
                
    p1, p2, p3, p4, p5 = process_it(galaxy, sampler, emcee_params, ss,\
                                    dt_blobs, fig_dir, std)
    print(galaxy.Galaxy, p1, p2, p3, p4, p5)
    print("acceptance", galaxy.emcee.acceptance, "   Run time: ",time()-t)
    print("min chisq/dof: ", min(galaxy.emcee.chisq)/(len(galaxy.Data.R)-1))
    if galaxy.Vflat > 0:
        a = galaxy.emcee.vmax/galaxy.Vflat
        print('<Vmax>/Vflat',np.percentile(a,[16, 50, 84],axis=0))
        a = galaxy.emcee.sigma0/galaxy.Vflat
        print('<sigma0>/Vflat',np.percentile(a,[16, 50, 84],axis=0))
    a = galaxy.emcee.vmax/galaxy.emcee.sigma0
    print('<Vmax/sigma0>',np.percentile(a,[16, 50, 84],axis=0))
    a = np.log10(27*(galaxy.emcee.vmax/100)**1.4/galaxy.emcee.rmax)/0.11
    print('<delta rmax>',np.percentile(a,[16, 50, 84],axis=0))
    a = np.array([get_m200_c200(v,r)[2]/0.11\
                  for v,r in zip(galaxy.emcee.vmax,galaxy.emcee.rmax)])
    print('<delta c>',np.percentile(a,[16, 50, 84],axis=0))
    a = galaxy.emcee.r1
    print('<r1>, rlast', np.percentile(a,[16, 50, 84],axis=0), galaxy.Data.R[-1])
    a = ss.rate_const*galaxy.emcee.rho0*galaxy.emcee.sigma0
    print('rate',np.percentile(a,[16, 50, 84],axis=0))
    print('SI time, dyn time (Gyr)', 10/np.median(a),\
          1/np.sqrt(ss.GNewton*np.median(galaxy.emcee.rho0)))
    print('----------------------------------------')

if save_output:
    with open(file_name_store,'wb') as file_object:
        pickle.dump(galaxies,file_object)
    print('pickled to "{}"'.format(file_name_store))
    file_object.close()

Checks from here on. Main cell to run the galaxies is above.

In [ ]:
ll = [min(np.min(g.emcee.chisq)/(len(g.Data.R)-1),10) for g in galaxies]
plt.hist(ll,cumulative=True,density=True)
plt.show()

mll = [np.median(g.emcee.ml_disk) for g in galaxies]
plt.hist(mll)
plt.show()

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
def select(galaxy):
    return True if len(galaxy.Data.R)>5 else False
mstar = [np.median(galaxy.emcee.ml_disk/galaxy.emcee.mvir)*1e9*galaxy.Lum \
         for galaxy in galaxies if select(galaxy)]
mhalo = [np.std(galaxy.emcee.mvir)/np.median(galaxy.emcee.mvir) \
         for galaxy in galaxies if select(galaxy)]
plt.plot(mhalo,mstar,'o')
plt.xscale('log')
plt.yscale('log')
plt.show()

minchisq = [np.min(galaxy.emcee.chisq)/(len(galaxy.Data.R)-\
        (5 if sum(galaxy.Data.Bulge)>0 else 4)) for galaxy in galaxies \
            if select(galaxy)]
plt.hist(np.log10(minchisq),cumulative=True,density=True,bins=20)

for galaxy in galaxies:
    print(galaxy.Galaxy if not select(galaxy) else "")
    if np.median(galaxy.emcee.ml_disk/galaxy.emcee.mvir)*1e9*galaxy.Lum<1e-3 and\
    np.std(galaxy.emcee.mvir)/np.median(galaxy.emcee.mvir) < 100:\
        print(galaxy.Galaxy,np.median(galaxy.emcee.ml_disk)*galaxy.Lum*1e9,\
              np.median(galaxy.emcee.mvir),np.min(galaxy.emcee.chisq),select(galaxy))
        

In [ ]:
import autocorr
reload(autocorr)
from autocorr import *
N, gw2010, new = get_autocorr(sampler)
plot_autocorr(N, (gw2010,), ("New",))

In [ ]:
import matplotlib.pyplot as plt
import corner
%matplotlib inline
rr = np.log10(galaxy.emcee.rho0*galaxy.emcee.sigma0*\
              galaxy.emcee.cross*(1.5278827817856099e-26*3.15576e+16*10))
plt.hist(rr, bins=np.linspace(bounds[0][0],bounds[0][1],20), normed=False, alpha=0.5)
plt.show()

params=list(zip(galaxy.emcee.slope_15pRvir, np.log10(galaxy.emcee.rho0),\
                rr,\
                np.log10(galaxy.emcee.r1),\
                np.log10(galaxy.emcee.sigma0),\
                galaxy.emcee.cvir*(galaxy.emcee.mvir/1e12)**0.09\
               ))
labels = ["slope","log10(rho0)","rr", "log10(r1)", "log10(sigma0)", "cv"]
fig = corner.corner(params,  show_titles=True, title_kwargs={"fontsize": 12}, labels=labels)
fig.show()

In [ ]:
red_chisq = galaxy.emcee.chisq/(len(galaxy.Data.V)-4)
i = np.argmin(red_chisq)
min_red_chisq = red_chisq[i]
min_red_chisq_params = galaxy.emcee.samples[i]
print(min_red_chisq, min_red_chisq_params)
print(galaxy.emcee.rho0[i]/1e8)
print(galaxy.emcee.vmax[i])
print(galaxy.emcee.rmax[i])
print(galaxy.emcee.sigma0[i])
print(np.percentile(galaxy.emcee.ml_bulge,(16,50,84)))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
pa = []
for galaxy in galaxies:
    a = np.array([get_m200_c200(v,r)[2]/0.11\
              for v,r in zip(galaxy.emcee.vmax,galaxy.emcee.rmax)])
    plt.scatter(np.median(a),np.median(np.log10(galaxy.emcee.ml_disk)))
plt.show()

In [ ]:
def check_prior(has_bulge):
    from SPARCdata import read_SPARC_data, dt, create_galaxies_arr, fill_galaxy, unpack_emcee_params
    emcee_params = ndim, nwalkers, nburn, niter, nthin, nthreads =\
    5, 40, 400, 2000, 5, 1
    n_poly = 4
    data, dt_in = read_SPARC_data(ddir)
    dt_galaxy, dt_blobs = dt(emcee_params, dt_in)
    fake = create_galaxies_arr(data, 0, 1, dt_galaxy)
    ss = sidm_setup()
    mn = match_nfw()
    for f in fake:
        fill_galaxy(ddir, f, n_poly)
        f.Galaxy += '_fake'
        f.Data.e_V = f.Data.V*1e10
        if has_bulge: f.Data.Bulge += 1
        prior_params, reg_params, bounds = get_priors_bounds(f)
        ndim, nwalkers, nburn, niter, nthin, nthreads = unpack_emcee_params(emcee_params)
        args = (f, ss, mn, emcee_params, prior_params, reg_params, bounds)
        start, std = start_pos(args)
        pos0 = np.array([np.array(start)+ [np.random.uniform(-stdv,stdv) for stdv in std] \
                         for i in range(nwalkers)])
        assert pos0.shape == (nwalkers, ndim)
        sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, args=[args], threads=nthreads)
        for pos, lnprob0, state, blobs in sampler.sample(pos0, iterations=nburn): # burn in
            pass
        sampler.reset() 
        sampler.run_mcmc(pos, niter)
        assert sampler.chain.shape == (nwalkers, niter, ndim)    
        p1, p2, p3, p4, p5 = process_it(f, sampler, emcee_params, ss,\
                                        dt_blobs, fig_dir, std)
    return fake, (p1, p2, p3, p4, p5)

check, p = check_prior(has_bulge=True)
print(p)

for g in check:
    print(g.Galaxy,sum(g.Data.Bulge))
    a = [get_m200_c200(v,r)[2]/0.11 for v,r in zip(g.emcee.vmax,g.emcee.rmax)]
    plt.hist(a)
    plt.show()

In [ ]:

# fieldnames_to_remove = []
# reduced_data = galaxies[ [ name for name in galaxies.dtype.names if name not in fieldnames_to_remove ] ]
# reduced_data = reduced_data.view(np.recarray)

reduced_data = galaxies.copy()

# reduced_data_2 = pickle.load( open("sparc_reduced_data_v1_save.txt","rb") )
# reduced_data_2 = reduced_data_2.view(np.recarray)
# reduced_data = stack_arrays((reduced_data_2[:47],galaxies[47:]),usemask=False,asrecarray=True)

run_index = 1

try:
    run_index
except NameError:
    run_index = 1
    
file_name_store = 'sparc_reduced_data_v%s.txt' % run_index 

while os.path.isfile(file_name_store):
    print('file "{}" exists'.format(file_name_store))
    run_index += 1
    file_name_store = 'sparc_reduced_data_v%s.txt' % run_index 
    print('trying new file named "{}"'.format(file_name_store))
try:
    with open(file_name_store,'wb') as file_object:
        pickle.dump(reduced_data,file_object)
except IOError as e:
    print('Unable to open file "{}"'.format(file_name_store)) #Does not exist OR no read permissions
print('pickled to "{}"'.format(file_name_store))
file_object.close()

del reduced_data

In [ ]:
import pickle, math
ss = sidm_setup()
file_name_store='sparc_cross_3_constrained.txt'
reduced_data = pickle.load( open(file_name_store,"rb") )
galaxies = reduced_data.view(np.recarray)
s = np.ravel(reduced_data.emcee.rho_r1*reduced_data.emcee.sigma0*reduced_data.emcee.cross*ss.rate_const)
ok = np.all([math.isclose(item, 1.0, rel_tol = 1e-4, abs_tol = 0.0) for item in s])
print(ok)
del reduced_data

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
fig, ax = plt.subplots(4, figsize=(10,20))
b=[]
def fc(v,r):
    return np.log10(27*(v/100)**1.4/r)/0.11
for galaxy in galaxies:
    a=fc(galaxy.emcee.vmax,galaxy.emcee.rmax)
    b.append([np.median(a),np.std(a)])
b=np.array(b)
ax[0].scatter(b[:,0],b[:,1])
ax[0].set_ylim([0,2])
ax[0].set_xlabel('Median of log10(rmax(Dutton)/rmax)/0.11')
ax[0].set_ylabel('Std Dev of log10(rmax(Dutton)/rmax)/0.11')

ax[1].scatter([np.median(galaxy.emcee.vmax) for galaxy in galaxies],\
              [fc(np.median(galaxy.emcee.vmax),np.median(galaxy.emcee.rmax))\
               for galaxy in galaxies])
ax[1].set_xlim([10,200])
ax[1].set_xlabel('Median Vmax')
ax[1].set_ylabel('log10(rmax(Dutton)/(median rmax))/0.11')

ml=[np.median(galaxy.emcee.ml_disk) for galaxy in galaxies]
ax[2].hist(ml)
ax[2].set_xlabel('M/L (disk)')

cc=[min(min(galaxy.emcee.chisq)/(len(galaxy.Data.V)-3),10) for galaxy in galaxies]
ax[3].hist(cc)
ax[3].set_xlabel('Chisq/dof')
fname = os.path.join(os.getcwd(), "sidm_3.png")
plt.savefig(fname)
plt.close(fig)


In [ ]:
for galaxy in galaxies:
    print(((galaxy.MHI*1e9+galaxy.Lum*1e9*0.5)/50)**(1/4))

In [ ]:
def dc(v,r):
    return np.abs(np.log10((v/100)**1.4*27/r)/0.11)
def chisq_by_conc(g):
    a=[(c//(len(g.Data.V)-4),dc(v,r)) for c,v,r in\
       zip(g.emcee.chisq,g.emcee.vmax,g.emcee.rmax)]
    return min([a1[0] for a1 in a if 0<a1[1]<1]+[-1]),\
           min([a1[0] for a1 in a if 1<a1[1]<2]+[-1]),\
           min([a1[0] for a1 in a if 2<a1[1]<3]+[-1]),\
           min([a1[0] for a1 in a if 3<a1[1]<4]+[-1]),\
           min([a1[0] for a1 in a if 4<a1[1]<5]+[-1]),\
           min([a1[0] for a1 in a if 5<a1[1]]+[-1])
for g in galaxies:
    print([(c,dc(v,r)) for c,v,r in\
       zip(g.emcee.chisq/(len(g.Data.V)-4),g.emcee.vmax,g.emcee.rmax)])
    print(chisq_by_conc(g))

In [ ]:
galaxies = create_galaxies_arr(data, 0, 135, dt_galaxy)
for galaxy in galaxies:
    fill_galaxy(ddir, galaxy, n_poly)

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
for galaxy in galaxies:
    plt.scatter(galaxy.Data.R[0]/galaxy.Data.R[-1],galaxy.Data.V[0]/galaxy.Vflat)
plt.xscale('log')
plt.yscale('log')
plt.show()

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
a=np.array([(np.log10(g.Data.R[-1]/g.Data.R[0]),g.Vflat) for g in galaxies])
plt.hist(a[:,0])
print([g.Galaxy for g in galaxies if np.log10(g.Data.R[-1]/g.Data.R[0])>1.5])

In [ ]:
for galaxy in galaxies:
    a = galaxy.emcee.sigma0/galaxy.Vflat
    print('<sigma0>/Vflat',np.percentile(a,[16, 50, 84],axis=0))


In [ ]:
for galaxy in galaxies:
    v = galaxy.emcee.vmax/galaxy.Vflat
    s = galaxy.emcee.sigma0/galaxy.Vflat
    if np.std(v) > 1: 
        print(galaxy.Galaxy,np.std(v),np.percentile(v,[16, 50, 99],axis=0),np.percentile(s,[5, 16],axis=0))


In [ ]:
for i, galaxy in enumerate(galaxies):
    a= galaxy.Data.e_V[galaxy.Data.e_V < 1]
    if len(a) > 0: print(i, galaxy.Galaxy, len(a), len(galaxy.Data.V))
#         print(list(zip(galaxy.Data.R,galaxy.Data.V,galaxy.Data.e_V)))

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
for galaxy in galaxies[mask(galaxies)]:
    a=np.log10(27*(galaxy.emcee.vmax/100)**1.4/galaxy.emcee.rmax)/0.11
    print(galaxy.Galaxy, np.median(a))
#     plt.scatter(np.median(a),np.std(a))
# plt.ylim([0,2])
# plt.show()